In [13]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble  import RandomForestClassifier
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter,
                                        UniformFloatHyperparameter, 
                                        UniformIntegerHyperparameter
from ConfigSpace.conditions import InCondition
from smac.scenario.scenario import Scenario
from smac.facade.smac_hpo_facade import SMAC4HPO
from smac.runhistory import runhistory2epm, runhistory


In [6]:
##Create SMAC's search space
cs = ConfigurationSpace()       
alg_name = CategoricalHyperparameter("alg_name", ["SVM", "RF"], default_value="SVM")
cs.add_hyperparameter(alg_name)  
##Search Space for Classifier: SVM
kernel = CategoricalHyperparameter("kernel", ["linear", "rbf", "poly", "sigmoid"], default_value="linear")
C = UniformFloatHyperparameter("C", 1e-2, 100, default_value=1.0)
degree = UniformIntegerHyperparameter("degree", 2, 50, default_value=3)     # Only used by kernel poly
coef0 = UniformFloatHyperparameter("coef0", 0.0, 10.0, default_value=0.0)  # poly, sigmoid
use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef0 = InCondition(child=coef0, parent=kernel, values=["poly", "sigmoid"])
gamma = UniformFloatHyperparameter("gamma", 0, 20, default_value=1)
##add hyperparameters into search space
cs.add_hyperparameters([kernel, C,coef0, degree, gamma])
cs.add_conditions([use_degree, use_coef0])
cs.add_condition(InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"]))
cs.add_condition(InCondition(child=kernel, parent=alg_name, values=['SVM']))
cs.add_condition(InCondition(child=C, parent=alg_name, values=['SVM']))    
##Search Space for Classifier: SVM

n_estimators = UniformIntegerHyperparameter("n_estimators", 5, 100, default_value=10)
criterion = CategoricalHyperparameter("criterion", ["gini", "entropy"], default_value="gini")
max_depth = UniformIntegerHyperparameter("max_depth", 10, 200, default_value=10)
max_features = CategoricalHyperparameter("max_features", ['auto', 'sqrt','log2','none'], default_value="auto")
cs.add_hyperparameters([n_estimators,criterion,max_depth, max_features])
cs.add_condition(InCondition(child=n_estimators, parent=alg_name, values=["RF"]))
cs.add_condition(InCondition(child=criterion, parent=alg_name, values=["RF"]))
cs.add_condition(InCondition(child=max_depth, parent=alg_name, values=["RF"]))
cs.add_condition(InCondition(child=max_features, parent=alg_name, values=["RF"]))


max_features | alg_name in {'RF'}

In [7]:
cs

Configuration space object:
  Hyperparameters:
    C, Type: UniformFloat, Range: [0.01, 100.0], Default: 1.0
    alg_name, Type: Categorical, Choices: {SVM, RF}, Default: SVM
    coef0, Type: UniformFloat, Range: [0.0, 10.0], Default: 0.0
    criterion, Type: Categorical, Choices: {gini, entropy}, Default: gini
    degree, Type: UniformInteger, Range: [2, 50], Default: 3
    gamma, Type: UniformFloat, Range: [0.0, 20.0], Default: 1.0
    kernel, Type: Categorical, Choices: {linear, rbf, poly, sigmoid}, Default: linear
    max_depth, Type: UniformInteger, Range: [10, 200], Default: 10
    max_features, Type: Categorical, Choices: {auto, sqrt, log2, none}, Default: auto
    n_estimators, Type: UniformInteger, Range: [5, 100], Default: 10
  Conditions:
    C | alg_name in {'SVM'}
    coef0 | kernel in {'poly', 'sigmoid'}
    criterion | alg_name in {'RF'}
    degree | kernel in {'poly'}
    gamma | kernel in {'rbf', 'poly', 'sigmoid'}
    kernel | alg_name in {'SVM'}
    max_depth | alg_n

In [19]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

def obj_func(params):
    params = {k : params[k] for k in params if params[k]}
    classifier = params['alg_name']
    params.pop("alg_name", None)    
    clf = SVC()
    if(classifier == 'SVM'):
        clf = SVC(**params)
    elif(classifier == 'RF'):
        clf = RandomForestClassifier(**params)
    mean= cross_val_score(clf, X, y).mean()
    loss = 1 - mean
    #print (mean)
    return loss

In [20]:
output_dir='./SMAC_blog_log/'
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 50,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "output_dir":output_dir,
                     "deterministic": "false"
                    })
smac = SMAC4HPO(scenario=scenario,rng=np.random.RandomState(1),tae_runner=obj_func)
incumbent = smac.optimize()

INFO:smac.utils.io.cmd_reader.CMDReader:Output to ./SMAC_blog_log/
INFO:smac.initial_design.default_configuration_design.DefaultConfiguration:Running initial design for 1 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0200
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 2 runs: 0.0200
INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 0.463638 sec, available: 0.000010 sec)
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 3 runs: 0.0200
INFO:smac.intensification.intensification.Intensifier:Wallclock time 

In [21]:
incumbent

Configuration:
  C, Value: 1.0
  alg_name, Value: 'SVM'
  kernel, Value: 'linear'